In [1]:
import pandas as pd
import gensim

In [2]:
data = pd.read_pickle('../../data/news/preprocessed_data.pkl')

In [3]:
len(data)

50332

# Dictionary encoding

In [7]:
keyword_data = data.키워드.values[:1000] #위치, 키워드 가능하나 일단은 '위치'

In [8]:
dictionary = gensim.corpora.Dictionary(keyword_data)
corpus = [dictionary.doc2bow(cnt) for cnt in keyword_data]

# Topic model (LDA)

In [9]:
topic_number = 10
number_of_passes = 15  # 기본값 1
number_of_iterations = 50  # 기본값 50
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic_number, id2word=dictionary,
                                            passes=number_of_passes,  iterations=number_of_iterations)

In [10]:
lda_model.print_topics()

[(0,
  '0.016*"매수" + 0.013*"대비" + 0.010*"주가" + 0.009*"외국인" + 0.009*"기관" + 0.009*"상승" + 0.008*"공개" + 0.007*"개인" + 0.007*"종목" + 0.006*"기록"'),
 (1,
  '0.014*"부동산" + 0.010*"분양" + 0.008*"결정" + 0.008*"주택" + 0.007*"청약" + 0.007*"공시" + 0.006*"신청" + 0.005*"토지" + 0.005*"지역" + 0.004*"상가"'),
 (2,
  '0.008*"사업" + 0.006*"예정" + 0.006*"입찰" + 0.005*"적용" + 0.004*"대림산업" + 0.004*"발행" + 0.004*"보험" + 0.004*"터키" + 0.004*"정부" + 0.004*"항공기"'),
 (3,
  '0.007*"시장" + 0.006*"선정" + 0.005*"사업" + 0.004*"매출" + 0.004*"경매" + 0.004*"증가" + 0.004*"규모" + 0.003*"콘크리트" + 0.003*"예정" + 0.003*"연체"'),
 (4,
  '0.010*"투자" + 0.010*"가구" + 0.007*"수익" + 0.007*"오피스텔" + 0.006*"주택" + 0.006*"건물" + 0.006*"금리" + 0.005*"신용" + 0.005*"가능" + 0.005*"최대"'),
 (5,
  '0.036*"가격" + 0.015*"아파트" + 0.011*"평균" + 0.010*"상승" + 0.009*"서울" + 0.008*"인상" + 0.007*"강남" + 0.006*"계란" + 0.006*"2년" + 0.005*"상승세"'),
 (6,
  '0.013*"대출" + 0.010*"가계" + 0.008*"가격" + 0.007*"은행" + 0.006*"인상" + 0.005*"증가" + 0.004*"금리" + 0.004*"정부" + 0.004*"소득" + 0.003*"자영업자"'),
 (7,
  '0.021*

# 시각화

In [ ]:
import pyLDAvis.gensim

In [ ]:
pyLDAvis.enable_notebook()
plot_lda_model = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(plot_lda_model)

# Topics in Docs

In [11]:
show_top_n = 10

print('  Topic Rates')
for cnt, topic_rate in enumerate(lda_model[corpus]):
    if cnt == show_top_n:
        break
    print('{0:>3}th article  :  {1}'.format(cnt, topic_rate))

  Topic Rates
  0th article  :  [(1, 0.033009294), (5, 0.7638015), (7, 0.2000607)]
  1th article  :  [(8, 0.37970498), (9, 0.6147366)]
  2th article  :  [(2, 0.13502392), (6, 0.6511889), (7, 0.08689104), (9, 0.12244486)]
  3th article  :  [(2, 0.88159716), (8, 0.11414347)]
  4th article  :  [(9, 0.997404)]
  5th article  :  [(5, 0.99399763)]
  6th article  :  [(8, 0.9956446)]
  7th article  :  [(2, 0.99603355)]
  8th article  :  [(0, 0.26033932), (6, 0.7362523)]
  9th article  :  [(9, 0.99513245)]


In [12]:
def make_topic_data(model, corpus):
    '''
    문서별 최상의 topic을 칼럼으로 가지는 pandas.DataFrame 반환
    model : gensim.models.ldamodel.LdaModel
    corpus : corpus
    '''
    topic_data = pd.DataFrame()

    for cnt, topic_rate_list in enumerate(model[corpus]):
        sort_topic_rate_list = sorted(
            topic_rate_list, key=lambda x: x[1], reverse=True)
        (top_topic_numb, top_topic_rate) = sort_topic_rate_list[0]
        
        topic_data = topic_data.append(pd.Series(
            [int(top_topic_numb), top_topic_rate, sort_topic_rate_list]), ignore_index=True)

    topic_data.columns = ['name_top_topic', 'rate_top_topic', 'topic_detil']
    return topic_data

In [13]:
topic_data = make_topic_data(lda_model, corpus)

In [14]:
topic_data[:10]

,name_top_topic,rate_top_topic,topic_detil
0,5.0,0.763802,"[(5, 0.7638015), (7, 0.20006076), (1, 0.033009..."
1,9.0,0.614734,"[(9, 0.6147343), (8, 0.37970737)]"
2,6.0,0.651189,"[(6, 0.65118915), (2, 0.13502301), (9, 0.12244..."
3,2.0,0.881603,"[(2, 0.8816033), (8, 0.11413733)]"
4,9.0,0.997404,"[(9, 0.997404)]"
5,5.0,0.993998,"[(5, 0.99399763)]"
6,8.0,0.995645,"[(8, 0.9956446)]"
7,2.0,0.996034,"[(2, 0.99603355)]"
8,6.0,0.736253,"[(6, 0.73625255), (0, 0.26033908)]"
9,9.0,0.995132,"[(9, 0.99513245)]"


In [24]:
data.iloc[:10,].reset_index()

,뉴스 식별자,일자,언론사,제목,통합 분류1,통합 분류2,위치,기관,키워드,특성추출(가중치순 상위 50개),URL
0,1.500301e+06,2017-01-31,경상일보,"[현장&이슈]천상 주택가 한가운데 주상복합,주민 반발",지역>경남,경제>부동산,"[울산, 범서읍, 백문현, 천상, 울주군]",(주)평강,"[천상, 주택가, 주상, 복합, 주민, 반발, 울산, 울주군, 범서읍, 천상, 주택...","[주민들, 주상복합, 시행사, 울주군, 탑마트, 주택가, 일조권, 신한다솜아파트, ...",www.ksilbo.co.kr/news/articleView.html?idxno=5...
1,1.500301e+06,2017-01-31,경상일보,울산 주택 매매 전세 월세 ‘트리플 약세’,경제>부동산,경제>유통,"[울산, 충북, 대전, 전북, 경기, 부산, 제주, 대구, 광주, 경남]","울산,감정원,한국감정원","[매매, 울산, 주택, 전세, 월세, 트리플, 약세, 새해, 울산, 주택, 매매, ...","[울산, 월세가격, 전세가격, 3개월, 하락세, 대구, 경남, 충북, 7만, 내림세...",www.ksilbo.co.kr/news/articleView.html?idxno=5...
2,1.500301e+06,2017-01-31,경상일보,"경남지역 아파트, 횡령 부당입찰 비리 여전",지역>부산,경제>부동산,"[회계, 창원지역, 경남지역, 양산지역, 경남도]","2억여원,국민연,공정거래위원회,경남도","[경남, 지역, 아파트, 횡령, 부당입찰, 비리, 여전, 경남지역, 아파트, 단지,...","[가입자, 관리비, 경남도, 양산, 교체공사, 과태료, 사업자, 10건, 회계담당자...",www.ksilbo.co.kr/news/articleView.html?idxno=5...
3,1.500301e+06,2017-01-31,경상일보,울산 북구 효문공단 잔여지 40여년만에 개발 재개,경제>부동산,지역>충남,"[울산, 울산미포국가산업단지, 북구, 울산시, 경기]","국토교통부,중앙토지수용위원회,효문공단,LH,울산시,수자원공사","[40여년, 울산, 북구, 효문, 공단, 잔여, 재개, 40년, 장기간, 방치, 울...","[효문공단, lh, 토지보상, 그동안, 울산시, 관계자, 장기간, 울산, 잔여지, ...",www.ksilbo.co.kr/news/articleView.html?idxno=5...
4,1.101101e+06,2017-01-31,한국일보,11ㆍ3대책에 얼었던 강남4구 아파트 다시 꿈틀,경제>부동산,경제>증권_증시,"[서울, 강남권, 강남구, 구로, 강남, 강남 3구, 신반포, 개포동, 비강남권, ...","송파구,리서치센터장,아너힐즈,정부,강남권,강남,한국감정원,도시계획위원회,서초구,반포...","[11ㆍ3대책, 강남, 아파트, 강남, 대책, 하락, 12주, 반등, 강남권, 11...","[강남, 강남권, 서울, 분양가, 신반포, 강남구, 잠원동, 부동산, 하락세, 12...",http://hankookilbo.com/v/f8021f7019024e87b693d...
5,1.101101e+06,2017-01-31,한국일보,강남 3구 재건축 아파트 2년새 3억원 급등,경제>부동산,경제>유통,"[미국, 동대문구, 서울, 용산구, 강남구, 중랑구, 강남, 노원구, 강남 3구, ...","강남구,부동산114리서치센터,부동산114","[2년, 강남, 아파트, 3억, 급등, 서울, 아파트, 가격, 상승세, 강남, 강남...","[3억, 강남, 서울, 부동산, 재건축, 송파구, 상승세, 10억, 1억, 858만...",http://hankookilbo.com/v/6a43ced7df33481cb8341...
6,1.101101e+06,2017-01-31,한국일보,"기아차, ‘스토닉’ 가격 및 사양 ‘프라이드’로 보여",경제>자동차,경제>부동산,"[개성, 미국, 하단, 영국, 유럽]","스포츠유틸리티차,자인,US PTO,유럽,스토닉,KED,카스쿠프스,특허청","[기아차, 스토닉, 가격, 사양, 프라이드, 기아자동차, 국내시장, 소형, 스포츠유...","[기아차, 리터, 스토닉, 영국, 가솔린, 특허청, 모터쇼, 축거, 하단부, 1만,...",http://hankookilbo.com/v/665c325912a74317956bc...
7,1.101101e+06,2017-01-31,한국일보,4월부터 장기 저축성보험 비과세 혜택 확 줄어든다,경제>금융_재테크,경제>부동산,"[아시아, 신성, 경기]","정부,진흥지구,국무회의,정부서울청사","[4월, 장기, 저축, 보험, 혜택, 일시납, 1억, 축소, 적립식, 제한, 중고,...","[사업비, 적립식, 보험료, 일시납, 다이오드, 판매업, 그간, 개정안, 가입분, ...",http://hankookilbo.com/v/e7d6dda333604b1ea08dc...
8,1.101101e+06,2017-01-31,한국일보,‘200만원 터치’ 삼성전자 액면분할 재점화,경제>부동산,경제>무역,[None],"유가증권,리서치본부장,미래전략실","[200만, 터치, 재점화, 삼성전자, 액면, 분할, 개인, 1주, 가격, 솔솔, ...","[액면분할, 삼성전자, 황제주, 200만, 투자자, 가능성, 관계자, 원선, 이창목...",http://hankookilbo.com/v/83f9efbc8b044ec6b91f2...
9,1.101101e+06,2017-01-31,한국일보,서울 강남에 아파트 1만8천가구 쏟아진다 11년만에 최대,경제>부동산,경제>유통,"[서초동, 서초우성1단지, 서울, 청담동, 강남구, 개포시영, 개포, 주공8단지, ...","주공7단지,주택도시보증공사,공무원연금관리공단","[서울, 강남, 아파트, 가구, 11년, 최대, 서울, 강남, 서초, 송파, 강동구...","[강남, 1만, 강남구, 서울, 부동산, 일반분양, 고덕, 강동구, 잠원동, 강남권...",http://hankookilbo.com/v/53e139a28e2340d58208f...


# join with original data

In [35]:
merge_data = pd.concat([data.iloc[:1000, ].reset_index(), topic_data], axis=1)[['뉴스 식별자', '일자', '언론사', '제목', '위치', 'URL',
                                                                                'name_top_topic', 'rate_top_topic', 'topic_detil']]

In [39]:
# key 추가
merge_data['year'] = merge_data.일자.dt.year
merge_data['month'] = merge_data.일자.dt.month
merge_data['day'] = merge_data.일자.dt.day
merge_data['week'] = merge_data.일자.dt.weekofyear

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [40]:
merge_data.head(6)

,뉴스 식별자,일자,언론사,제목,위치,URL,name_top_topic,rate_top_topic,topic_detil,year,month,day,week
0,1.500301e+06,2017-01-31,경상일보,"[현장&이슈]천상 주택가 한가운데 주상복합,주민 반발","[울산, 범서읍, 백문현, 천상, 울주군]",www.ksilbo.co.kr/news/articleView.html?idxno=5...,5.0,0.763802,"[(5, 0.7638015), (7, 0.20006076), (1, 0.033009...",2017,1,31,5
1,1.500301e+06,2017-01-31,경상일보,울산 주택 매매 전세 월세 ‘트리플 약세’,"[울산, 충북, 대전, 전북, 경기, 부산, 제주, 대구, 광주, 경남]",www.ksilbo.co.kr/news/articleView.html?idxno=5...,9.0,0.614734,"[(9, 0.6147343), (8, 0.37970737)]",2017,1,31,5
2,1.500301e+06,2017-01-31,경상일보,"경남지역 아파트, 횡령 부당입찰 비리 여전","[회계, 창원지역, 경남지역, 양산지역, 경남도]",www.ksilbo.co.kr/news/articleView.html?idxno=5...,6.0,0.651189,"[(6, 0.65118915), (2, 0.13502301), (9, 0.12244...",2017,1,31,5
3,1.500301e+06,2017-01-31,경상일보,울산 북구 효문공단 잔여지 40여년만에 개발 재개,"[울산, 울산미포국가산업단지, 북구, 울산시, 경기]",www.ksilbo.co.kr/news/articleView.html?idxno=5...,2.0,0.881603,"[(2, 0.8816033), (8, 0.11413733)]",2017,1,31,5
4,1.101101e+06,2017-01-31,한국일보,11ㆍ3대책에 얼었던 강남4구 아파트 다시 꿈틀,"[서울, 강남권, 강남구, 구로, 강남, 강남 3구, 신반포, 개포동, 비강남권, ...",http://hankookilbo.com/v/f8021f7019024e87b693d...,9.0,0.997404,"[(9, 0.997404)]",2017,1,31,5
5,1.101101e+06,2017-01-31,한국일보,강남 3구 재건축 아파트 2년새 3억원 급등,"[미국, 동대문구, 서울, 용산구, 강남구, 중랑구, 강남, 노원구, 강남 3구, ...",http://hankookilbo.com/v/6a43ced7df33481cb8341...,5.0,0.993998,"[(5, 0.99399763)]",2017,1,31,5


# 부동산 가격 데이터와 병합